In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [0]:
from google.colab import files
src = list(files.upload().values())[0]
open('gp.py','wb').write(src)
import gp

%load gp.py
%run gp.py

Saving gp.py to gp.py


In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [0]:
train_images.shape

(60000, 28, 28)

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

In [0]:
s1 = "model =keras.Sequential([keras.layers.Flatten(input_shape=(28, 28)),"

s2 = ""
layers = 1
nodes = [128]
for i in range(1, layers +1):
  s2 = s2 + "keras.layers.Dense(" + str(nodes[i-1]) + ", activation=tf.nn.relu),"

s3 = "keras.layers.Dense(10, activation=tf.nn.softmax)])"

In [0]:
s = s1+s2+s3

In [0]:
exec(s)

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(train_images, train_labels, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 96us/sample - loss: 0.4955 - accuracy: 0.8254
Epoch 2/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.3780 - accuracy: 0.8636
Epoch 3/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.3369 - accuracy: 0.8768
Epoch 4/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.3100 - accuracy: 0.8868
Epoch 5/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.2935 - accuracy: 0.8912
Epoch 6/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.2807 - accuracy: 0.8953
Epoch 7/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.2678 - accuracy: 0.8995
Epoch 8/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.2575 - accuracy: 0.9042
Epoch 9/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.2474 - accuracy: 0.9071
Epoch 10/10
60

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

10000/1 - 1s - loss: 0.2405 - accuracy: 0.8841

Test accuracy: 0.8841


In [0]:
'''This is an attempt to design the complete architecture via Bayesian optimisation. But realised too complex for now. I will take it up during the semester break'''

'''def sample_loss_NN(params):
  h1 = np.int(params[0])
  h2 = list(map(np.int, params[1]))
  h3 = np.int(params[2])
  
  s1 = "model =keras.Sequential([keras.layers.Flatten(input_shape=(28, 28)),"
  s2 = ""
  layers = h1
  nodes = h2
  for i in range(1, layers +1):
    s2 = s2 + "keras.layers.Dense(" + str(nodes[i-1]) + ", activation=tf.nn.relu),"
  s3 = "keras.layers.Dense(10, activation=tf.nn.softmax)])"
  s = s1+s2+s3
  exec(s)
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  model.fit(train_images, train_labels, epochs=h3)

  return cross_val_score(model,X=train_images,y=train_labels,scoring='accuracy', cv=3).mean()'''

'def sample_loss_NN(params):\n  h1 = np.int(params[0])\n  h2 = list(map(np.int, params[1]))\n  h3 = np.int(params[2])\n  \n  s1 = "model =keras.Sequential([keras.layers.Flatten(input_shape=(28, 28)),"\n  s2 = ""\n  layers = h1\n  nodes = h2\n  for i in range(1, layers +1):\n    s2 = s2 + "keras.layers.Dense(" + str(nodes[i-1]) + ", activation=tf.nn.relu),"\n  s3 = "keras.layers.Dense(10, activation=tf.nn.softmax)])"\n  s = s1+s2+s3\n  exec(s)\n  model.compile(optimizer=\'adam\',\n              loss=\'sparse_categorical_crossentropy\',\n              metrics=[\'accuracy\'])\n  model.fit(train_images, train_labels, epochs=h3)\n\n  return cross_val_score(model,X=train_images,y=train_labels,scoring=\'accuracy\', cv=3).mean()'

In [0]:
layers = 1

In [0]:
def sample_loss_NN(params):
  s1 = "model =keras.Sequential([keras.layers.Flatten(input_shape=(28, 28)),"
  s2 = ""
  nodes = params
  for i in range(1, layers +1):
    s2 = s2 + "keras.layers.Dense(" + str(nodes[i-1]) + ", activation=tf.nn.relu),"
  s3 = "keras.layers.Dense(10, activation=tf.nn.softmax)])"
  s = s1+s2+s3
  exec(s)
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  model.fit(train_images, train_labels, epochs=10)
  
  return 0.65*model.evaluate(test_images,  test_labels, verbose=2)[1] + 0.35*model.evaluate(train_images,  train_labels, verbose=2)[1]


In [0]:
layers = 1

bounds = []
for j in range(1, layers+1):
  bounds.append([1,round(500/layers)])
bounds = np.array(bounds)
xp, yp = bayesian_optimisation(n_iters=25, sample_loss=sample_loss_NN, 
                              bounds=bounds,
                              n_pre_samples=10)
xp_hat1 = np.round(xp[np.array(yp).argmax(), :])

print(xp_hat1)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 96us/sample - loss: 0.0111 - accuracy: 0.9989
Epoch 2/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0066 - accuracy: 0.9992
Epoch 3/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0051 - accuracy: 0.9992
Epoch 4/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0136 - accuracy: 0.9987
Epoch 5/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0042 - accuracy: 0.9995
Epoch 6/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0105 - accuracy: 0.9987
Epoch 7/10
60000/60000 [==============================] - 5s 83us/sample - loss: 0.0046 - accuracy: 0.9994
Epoch 8/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0066 - accuracy: 0.9990
Epoch 9/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0090 - accuracy: 0.9989
Epoch 10/10
60

In [0]:
layers = 2 
bounds = []
for j in range(1, layers+1):
  bounds.append([1,round(1000/layers)])
bounds = np.array(bounds)
xp, yp = bayesian_optimisation(n_iters=25, sample_loss=sample_loss_NN, 
                              bounds=bounds,
                              n_pre_samples=10)
xp_hat2 = np.round(xp[np.array(yp).argmax(), :])

print(xp_hat2)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 90us/sample - loss: 0.0098 - accuracy: 0.9989
Epoch 2/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0026 - accuracy: 0.9995
Epoch 3/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0124 - accuracy: 0.9985
Epoch 4/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0044 - accuracy: 0.9994
Epoch 5/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0044 - accuracy: 0.9995
Epoch 6/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.0105 - accuracy: 0.9989
Epoch 7/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.0065 - accuracy: 0.9991
Epoch 8/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.0052 - accuracy: 0.9995
Epoch 9/10
60000/60000 [==============================] - 5s 77us/sample - loss: 0.0079 - accuracy: 0.9989
Epoch 10/10
60

In [0]:
layers = 3
bounds = []
for j in range(1, layers+1):
  bounds.append([5,round(500/layers)])
bounds = np.array(bounds)
xp, yp = bayesian_optimisation(n_iters=25, sample_loss=sample_loss_NN, 
                              bounds=bounds,
                              n_pre_samples=10)
xp_hat3 = np.round(xp[np.array(yp).argmax(), :])

print(xp_hat3)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 85us/sample - loss: 0.0079 - accuracy: 0.9989
Epoch 2/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0102 - accuracy: 0.9986
Epoch 3/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0042 - accuracy: 0.9994
Epoch 4/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0095 - accuracy: 0.9987
Epoch 5/10
60000/60000 [==============================] - 5s 79us/sample - loss: 0.0021 - accuracy: 0.9997
Epoch 6/10
60000/60000 [==============================] - 5s 86us/sample - loss: 0.0105 - accuracy: 0.9986
Epoch 7/10
60000/60000 [==============================] - 5s 88us/sample - loss: 0.0064 - accuracy: 0.9992
Epoch 8/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0100 - accuracy: 0.9986
Epoch 9/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0026 - accuracy: 0.9995
Epoch 10/10
60

In [0]:
layers =4 
bounds = []
for j in range(1, layers+1):
  bounds.append([1,round(300/layers)])
bounds = np.array(bounds)
xp, yp = bayesian_optimisation(n_iters=25, sample_loss=sample_loss_NN, 
                              bounds=bounds,
                              n_pre_samples=10)
xp_hat4 = np.round(xp[np.array(yp).argmax(), :])

print(xp_hat4)

'layers =4 \nbounds = []\nfor j in range(1, layers+1):\n  bounds.append([1,round(300/layers)])\nbounds = np.array(bounds)\nxp, yp = bayesian_optimisation(n_iters=25, sample_loss=sample_loss_NN, \n                              bounds=bounds,\n                              n_pre_samples=10)\nxp_hat4 = np.round(xp[np.array(yp).argmax(), :])\n\nprint(xp_hat4)'

In [0]:
layers =5 
bounds = []
for j in range(1, layers+1):
  bounds.append([5,round(300/layers)])
bounds = np.array(bounds)
xp, yp = bayesian_optimisation(n_iters=25, sample_loss=sample_loss_NN, 
                              bounds=bounds,
                              n_pre_samples=10)
xp_hat5 = np.round(xp[np.array(yp).argmax(), :])

print(xp_hat5)

'layers =5 \nbounds = []\nfor j in range(1, layers+1):\n  bounds.append([5,round(300/layers)])\nbounds = np.array(bounds)\nxp, yp = bayesian_optimisation(n_iters=25, sample_loss=sample_loss_NN, \n                              bounds=bounds,\n                              n_pre_samples=10)\nxp_hat5 = np.round(xp[np.array(yp).argmax(), :])\n\nprint(xp_hat5)'

In [0]:
layers =6 
bounds = []
for j in range(1, layers+1):
  bounds.append([1,round(500/layers)])
bounds = np.array(bounds)
xp, yp = bayesian_optimisation(n_iters=25, sample_loss=sample_loss_NN, 
                              bounds=bounds,
                              n_pre_samples=10)
xp_hat6 = np.round(xp[np.array(yp).argmax(), :])

print(xp_hat6)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 86us/sample - loss: 0.0076 - accuracy: 0.9989
Epoch 2/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0046 - accuracy: 0.9994
Epoch 3/10
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0055 - accuracy: 0.9992
Epoch 4/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.0079 - accuracy: 0.9990
Epoch 5/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0074 - accuracy: 0.9988
Epoch 6/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0041 - accuracy: 0.9993
Epoch 7/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0135 - accuracy: 0.9986
Epoch 8/10
60000/60000 [==============================] - 5s 89us/sample - loss: 0.0055 - accuracy: 0.9992
Epoch 9/10
60000/60000 [==============================] - 5s 88us/sample - loss: 0.0035 - accuracy: 0.9994
Epoch 10/10
60

The next steps are to run the best results from each layer and pick the best architecture for the NN model. So we do grid search for number of layers and Bayesian Optimisation for nodes per layer. I will take the optimising full architecture as next steps to this